In [2]:
import numpy as np
import time
from math import sin,cos,exp,log,pi
from SALib.sample import saltelli
from SALib.analyze import sobol
from SALib.test_functions import Ishigami
from numba import njit
import threading

In [3]:
C1=1.2
C2=0.3
C3=3
C4=pi+1
n=1_000_000



In [4]:

problem = {
    'num_vars' : 5,
    'names' : ['x1', 'x2', 'x3', 'x4','x5'],
    'bounds' : [[-pi,pi],
                [-pi,pi],
                [-pi,pi],
                [-pi,pi],
                [-pi,pi]
               ]
}

## Baseline 

In [5]:
def func(X):
    return C1*(X[0]*sin(X[0]))/X[3] + C2*sin(X[1]) + cos(exp(X[2]*X[2])) + C3*log(C4+X[4])

start = time.time()
param_values = saltelli.sample(problem, n)
print("samples generation took %s seconds" %(time.time() - start))

Y = np.zeros(param_values.shape[0])

start = time.time()
for i, X in enumerate(param_values):
    Y[i] = func(X)
print("model evaluation took %s seconds" %(time.time() - start))

start = time.time()
S = sobol.analyze(problem, Y)
print("SA took %s seconds" %(time.time() - start))

samples generation took 56.739498138427734 seconds
model evaluation took 31.06395697593689 seconds
SA took 314.30868005752563 seconds


In [6]:
S

{'S1': array([-5.15305236e-05,  1.65319429e-07,  6.84391709e-07,  2.38840430e-01,
         1.50205276e-05]),
 'S1_conf': array([1.66066181e-04, 6.45459699e-07, 2.61229359e-06, 3.17239255e-01,
        1.34538964e-05]),
 'ST': array([4.57063240e-01, 2.50683462e-07, 2.32440384e-06, 8.75035155e-01,
        1.36706656e-05]),
 'ST_conf': array([1.18749825e+00, 2.79975430e-07, 2.59743055e-06, 6.35954540e-01,
        1.52614129e-05]),
 'S2': array([[            nan,  3.76374620e-05,  3.61390483e-05,
          3.59468473e-01,  3.61617943e-05],
        [            nan,             nan, -4.44373076e-08,
          3.22967025e-09, -4.41339798e-08],
        [            nan,             nan,             nan,
          4.61198628e-06,  3.21385542e-06],
        [            nan,             nan,             nan,
                     nan,  4.71135765e-01],
        [            nan,             nan,             nan,
                     nan,             nan]]),
 'S2_conf': array([[           nan, 1.717

## Numba 

In [7]:
@njit(fastmath=True)
def func(X):
    return C1*(X[0]*sin(X[0]))/X[3] + C2*sin(X[1]) + cos(exp(X[2]*X[2])) + C3*log(C4+X[4])


start = time.time()
param_values = saltelli.sample(problem, n)
print("samples generation took %s seconds" %(time.time() - start))

Y = np.zeros(param_values.shape[0])

start = time.time()
for i, X in enumerate(param_values):
    Y[i] = func(X)
print("model evaluation took %s seconds" %(time.time() - start))

start = time.time()
S = sobol.analyze(problem, Y)
print("SA took %s seconds" %(time.time() - start))

samples generation took 56.46754193305969 seconds
model evaluation took 8.255743980407715 seconds
SA took 307.17661809921265 seconds


### Видно, что evalution заняло в 4-5 раз меньше чем без Numba
#### Попробуем ради интереса еще и распараллелить(результат не гарантируется)

In [8]:
@njit(fastmath=True,parallel=True)
def func(X):
    return C1*(X[0]*sin(X[0]))/X[3] + C2*sin(X[1]) + cos(exp(X[2]*X[2])) + C3*log(C4+X[4])


start = time.time()
param_values = saltelli.sample(problem, n)
print("samples generation took %s seconds" %(time.time() - start))

Y = np.zeros(param_values.shape[0])

start = time.time()
for i, X in enumerate(param_values):
    Y[i] = func(X)
print("model evaluation took %s seconds" %(time.time() - start))

start = time.time()
S = sobol.analyze(problem, Y)
print("SA took %s seconds" %(time.time() - start))

samples generation took 56.85862874984741 seconds


/Users/workhard/opt/anaconda3/lib/python3.7/site-packages/numba/compiler.py:602: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "<ipython-input-8-aa2b5931d905>", line 2:
@njit(fastmath=True,parallel=True)
def func(X):
^

  self.func_ir.loc))


model evaluation took 8.097552299499512 seconds
SA took 309.0211362838745 seconds


## Multithreads

In [10]:
n=1_000_000
nThreads=8

def func(X):
    return C1*(X[0]*sin(X[0]))/X[3] + C2*sin(X[1]) + cos(exp(X[2]*X[2])) + C3*log(C4+X[4])

def funcParallel(tid):
    lenParams=len(param_values)
    lenPart=int(lenParams/nThreads)
    start=tid*lenPart
    end=(tid+1)*lenPart
    #print(str(tid),str(lenParams),str(tid*lenPart),str((tid+1)*lenPart)+'\n')
    #print(tid,start,end,'\n')
    for i, X in enumerate(param_values[start:end]):
        Y[i+start] = func(X)

start = time.time()
param_values = saltelli.sample(problem, n)
print("samples generation took %s seconds" %(time.time() - start))

Y = np.zeros(param_values.shape[0])

start = time.time()
threads=[]
for tid in range(nThreads):
    th=threading.Thread(target=funcParallel,args=(tid,))
    threads.append(th)
    th.start()
    
for th in threads:
    th.join()
    #print(th)
            
print("parallel model evaluation took %s seconds" %(time.time() - start))

start = time.time()
S = sobol.analyze(problem, Y)
print("SA took %s seconds" %(time.time() - start))

samples generation took 56.243135929107666 seconds
parallel model evaluation took 30.96094584465027 seconds
SA took 319.53917026519775 seconds


#### К сожалению, мультитрединг не дал прироста скорости(прирост на уровне погрешности)